In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -U --no-cache-dir gdown --pre

In [ ]:
import gdown

gdown.download_folder(
    "https://drive.google.com/drive/folders/12orVtSezAOX_JublclhaWxP-f0PczpkT",
    output="./mvcgan",
    quiet=True
);

In [ ]:
import os

os.chdir('/content')
CODE_DIR = 'latentswap3d'

!git clone --recurse-submodules -j8 https://github.com/enisimsar/latentswap3d.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')

!pip install torch==1.10.0+cu102 torchvision==0.11.0+cu102 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

!pip install -r requirements.txt

import sys
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy as dc
from sklearn.metrics.pairwise import cosine_similarity

sys.path.append(".")

In [ ]:
device = "cuda"

from src.generators.mvcgan import MVCGANGenerator

In [ ]:
def generate_image(latent_codes):
    left_image = generator.synthesize(
        torch.from_numpy(latent_codes).to(generator.device), h_angle=-0.25, v_angle=0
    )[0]
    
    center_image = generator.synthesize(
        torch.from_numpy(latent_codes).to(generator.device), h_angle=0, v_angle=0
    )[0]
    
    right_image = generator.synthesize(
        torch.from_numpy(latent_codes).to(generator.device), h_angle=0.25, v_angle=0
    )[0]

    image = np.concatenate([left_image, center_image, right_image], axis=1)
    
    return image

## Initialize Models

In [ ]:
generator = MVCGANGenerator(device, class_name="FFHQ")

## Sample Face

In [ ]:
SEED = 272960755 # np.random.randint(np.iinfo(np.int32).max)

sample_code = (
    generator.get_codes(generator.sample_latent(1, SEED)).detach().cpu().numpy()
)

image = generate_image(sample_code)
plt.imshow(image)

## Load Directions

In [ ]:
BASE_DIR = "../mvcgan/"
latent_codes = np.load(os.path.join(BASE_DIR, "output.npz"))["latent_codes"]
attributes = pd.read_csv(os.path.join(BASE_DIR, "attributes.csv"))
feature_importances = json.load(open(os.path.join(BASE_DIR, "results.json")))
best_ks = json.load(open(os.path.join(BASE_DIR, "best_parameters.json")))

In [ ]:
df = pd.DataFrame()

sim = []
for i, _ in df.iterrows():
    sim.append(cosine_similarity(sample_code, latent_codes[i].reshape(1, -1))[0, 0])
df["sim"] = sim

## Editing

In [ ]:
attribute_dir = {
    "smiling": -1,
    "eyeglasses": -1,
    "facial_hair": 1
}
attr = 'facial_hair' #@param ["smiling", "eyeglasses", "facial_hair"]

In [ ]:
direction = attribute_dir[attr]
df["attribute"] = attributes[attr].values

positive_code = latent_codes[
    df.sort_values(["attribute", "sim"], ascending=[False, True]).index[0]
].reshape(1, -1)
negative_code = latent_codes[
    df.sort_values(["attribute", "sim"]).index[0]
].reshape(1, -1)

feature_importance = feature_importances[attr]
best_k = best_ks[attr]["best_k"]

codes = positive_code
if direction < 0:
    codes = negative_code

manipulated_code = dc(sample_code)
for ind, val in zip(
    feature_importance[:best_k], codes[:, feature_importance[:best_k]][0]
):
    manipulated_code[:, ind] = val

image = generate_image(manipulated_code)

plt.imshow(image)